In [1]:
import casadi as ca
import numpy as np

ref: https://www.researchgate.net/publication/299465495_An_Augmented_Lagrangian_Based_Algorithm_for_Distributed_NonConvex_Optimization

In [2]:
def create_prox_func(Nx,f_func):
    '''
    Not used here.
    '''
    x = ca.SX.sym("x",Nx)
    v = ca.SX.sym("v",Nx)
    lambda_ = ca.SX.sym("lambda",1)
    
    f = f_func(x)
    f_prox = f + 1/2 * lambda_ * ca.norm_2(x - v) ** 2
    f_prox_func = ca.Function("f_prox_func", [lambda_, x, v], [f_prox])

    p = ca.vertcat(v,lambda_)
    
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':x, 'f':f_prox, 'p': p}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

### Optimization fomulation

$\min _{x} \sum_{i=1}^{N} f_{i}\left(x_{i}\right) \quad$ s.t. $\quad\left\{\begin{array}{l}\sum_{i=1}^{N} A_{i} x_{i}=b \\ h_{i}\left(x_{i}\right) \leq 0, \quad i \in\{1, \ldots, N\}\end{array}\right.$

### Reformulated optimization problem

$\min _{x, y} \sum_{i=1}^{N} g_{i}\left(y_{i}\right)+I_{0}\left(\sum_{i=1}^{N} A_{i} x_{i}-b\right) \quad$ s.t. $\quad A_{i}\left(y_{i}-x_{i}\right)=0, \quad i \in\{1, \ldots, N\}$, where $I_{0}(r)=\left\{\begin{array}{ll}0 & \text { if } r=0 \\ \infty & \text { otherwise }\end{array}\right.$ and $\forall \in\{1, \ldots, N\}, \quad g_{i}\left(y_{i}\right)=\left\{\begin{array}{ll}f_{i}\left(y_{i}\right) & \text { if } h_{i}\left(y_{i}\right) \leq 0 \\ \infty & \text { otherwise }\end{array}\right.$ 

### Augmented Lagrangian function of the reformulated problem


\begin{equation}
L_{\rho}(x, y, \lambda)=I_{0}\left(\sum_{i=1}^{N} A_{i} x_{i}-b\right)+\sum_{i=1}^{N}\left\{g_{i}\left(y_{i}\right)+\lambda_{i}^{\top} A_{i}\left(y_{i}-x_{i}\right)+\frac{\rho}{2}\left\|A_{i}\left(y_{i}-x_{i}\right)\right\|^{2}\right\}
\end{equation}

$\begin{aligned} y^{k+1} &:=\underset{y}{\operatorname{argmin}} L_{\rho}\left(x^{k}, y, \lambda^{k}\right) \\ x^{k+1} &:=\underset{x}{\operatorname{argmin}} L_{\rho}\left(x, y^{k+1}, \lambda^{k}\right) \\ \lambda_{i}^{k+1}&=\lambda_{i}+\rho A_{i}\left(y_{i}-x_{i}\right) \end{aligned}$

$\lambda$ is the dual variable here. This update rule based on an augmented Lagrangian function could be reformulated into a proximal form. (Relation between ADMM and Augmented Lagrangians)

### Subproblem formulation

$\min _{y_{i}} f_{i}\left(y_{i}\right)+\lambda_{i}^{\top} A_{i} y_{i}+\frac{p}{2}\left\|A_{i}\left(y_{i}-x_{i}\right)\right\|_{2}^{2} \quad$ s.t. $\quad h_{i}\left(y_{i}\right) \leq 0$

In [3]:
def create_subproblem(fi_func, Ai, rho):
    N_lambda_i, N_yi = np.shape(Ai)
    yi = ca.SX.sym("yi",N_yi)
    xi = ca.SX.sym("xi",N_yi)
    lambda_i = ca.SX.sym("lambda_i",N_lambda_i)
    
    fi = fi_func(yi) + lambda_i.T @ Ai @ yi + rho/2 * ca.norm_2(Ai @ (yi - xi))**2 
    p = ca.vertcat(lambda_i, xi)
    g = yi
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':yi, 'g':g, 'f':fi, 'p': p}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

### Equality constrained QP problem
$\min _{x^{+}} \sum_{i=1}^{N}\left\{\frac{\rho}{2}\left\|A_{i}\left(y_{i}-x_{i}^{+}\right)\right\|_{2}^{2}-\left(\lambda_{i}^{+}\right)^{\top} A_{i} x_{i}^{+}\right\} \quad$ s.t. $\quad \sum_{i=1}^{N} A_{i} x_{i}^{+}=b$

In [4]:
def create_QP_problem(A_list,rho):
    N = len(A_list)
    x_plus_list = []
    y_i_list = []
    lambda_plus_list = []
    obj = 0
    g = 0
    for i in range(N):
        Ai = A_list[i]
        
        N_lambda_i, N_yi = np.shape(Ai)
        yi = ca.SX.sym("yi",N_yi)
        x_plus = ca.SX.sym("xi",N_yi)
        lambda_plus = ca.SX.sym("lambda_i",N_lambda_i)
        
        x_plus_list += [x_plus]
        y_i_list += [yi] 
        lambda_plus_list += [lambda_plus]
    
        obj += rho/2 * ca.norm_2(Ai @ (yi - x_plus))**2  - lambda_plus.T @ Ai @ yi
        g += Ai @ x_plus
    x = ca.vertcat(*x_plus_list)
    p = ca.vertcat(*(lambda_plus_list + y_i_list))
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':x, 'g':g, 'f':obj, 'p': p}
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver    

### Example 1
$\min _{x} x_{1} \cdot x_{2} \quad$ s.t. $\quad x_{1}-x_{2}=0$

analytical solution of $y$ is: $y=\left(\begin{array}{c}-2 \\ 2\end{array}\right) \lambda$

Numerical problem of ADMM with divergent $\lambda^{+} = - 2 \lambda$

In [5]:
eps = 1e-5
rho = 0.75
N_itermax = 50
A_list = []
fi_func_list = []

A = ca.DM([[1,-1]])
A_list += [A]
N = len(A_list)
b = ca.DM([0])

Nx = 2
x = ca.SX.sym("x",Nx)
fi_func = ca.Function("fi_func", [x], [x[0] * x[1]])
fi_func_list += [fi_func]

subsolver_list = []
# Define subproblem solvers
for i in range(N):
    Ai = A_list[i]
    fi_func = fi_func_list[i]
    subsolver_list += [create_subproblem(fi_func, Ai, rho)]
QP_solver = create_QP_problem(A_list, rho)
# Initial guess
xi_list = []
yi_list = []
lambda_i_list = []
lbx_list = []
ubx_list = []
for i in range(N):
    Ai = A_list[i]
    N_lambda_i, N_xi = np.shape(Ai)
    
    xi = np.random.randn(N_xi,1).flatten().tolist()
    lambda_i = np.random.randn(N_lambda_i,1).flatten().tolist()
    
    xi_list += [xi]
    lambda_i_list += [lambda_i]
    lbx_list += [[-ca.inf] * N_xi]
    ubx_list += [[ca.inf] * N_xi]
    yi_list += [[0] * N_xi]

nl_sub = {}

nl_qp = {}
nl_qp['lbg'] = b
nl_qp['ubg'] = b
nl_qp['lbx'] = sum(lbx_list,[])
nl_qp['ubx'] = sum(ubx_list,[])

# Track solution
yi_sol_list = []
x_plus_sol_list = []
lambda_i_sol_list = []
yi_sol_list += yi_list
x_plus_sol_list += [sum(xi_list,[])]
lambda_i_sol_list = [sum(lambda_i_list,[])]
for i in range(N_itermax):
    sum_Ay = 0
    for j in range(N):
        Ai = A_list[j]
        N_lambda_i, N_xi = np.shape(Ai)
        
        nl_sub['x0'] = yi_list[j]
        nl_sub['lbx'] = lbx_list[j]
        nl_sub['ubx'] = ubx_list[j]
        nl_sub['p'] = lambda_i_list[j] + xi_list[j]
        
        solver_subproblem = subsolver_list[j]
        yi_sol = solver_subproblem(**nl_sub)
        yi_list[j] = yi_sol['x'].full().flatten().tolist()
        yi_sol_list += [yi_list[j]]
        
        sum_Ay += Ai @ yi_sol['x']
        lambda_i_list[j] = (ca.DM(lambda_i_list[j]) + rho * Ai @ (ca.DM(yi_list[j]) - ca.DM(xi_list[j]))).full().flatten().tolist()
        lambda_i_sol_list += [lambda_i_list[j]]
    if ca.norm_1(sum_Ay - b) <= eps:
        break
    
    nl_qp['x0'] = sum(xi_list,[])    #  2D list to 1D
    nl_qp['p'] = sum(lambda_i_list,[]) + sum(xi_list,[])
    xi_sol = solver_subproblem(**nl_qp)
    # Update x_plus
    pos = 0
    for j in range(N):
        
        xi_plus_list = xi_sol['x'].full().flatten().tolist()
        list_len = len(xi_list[j])
        xi_list[j] = xi_plus_list[pos:pos+list_len]
        pos += list_len
    
    x_plus_sol_list += [xi_plus_list]


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [6]:
yi_sol_list

[[0, 0],
 [-2.772316007947563, 2.772316007947563],
 [5.544632015895125, -5.544632015895125],
 [-11.08926403179025, 11.08926403179025],
 [22.1785280635805, -22.1785280635805],
 [-44.357056127161, 44.357056127161],
 [88.714112254322, -88.714112254322],
 [-177.42822450864395, 177.42822450864395],
 [354.8564490172876, -354.8564490172876],
 [-709.7128980345753, 709.7128980345753],
 [1419.425796069151, -1419.425796069151],
 [-2838.8515921383023, 2838.8515921383023],
 [5677.703184276606, -5677.703184276606],
 [-11355.406368553211, 11355.406368553211],
 [22710.812737106426, -22710.812737106426],
 [-45421.625474212844, 45421.625474212844],
 [90843.2509484257, -90843.2509484257],
 [-181686.50189685138, 181686.50189685138],
 [363373.0037937028, -363373.0037937028],
 [-726746.0075874055, 726746.0075874055],
 [1453492.0151748112, -1453492.0151748112],
 [-2906984.030349622, 2906984.030349622],
 [5813968.060699245, -5813968.060699245],
 [-11627936.121398488, 11627936.121398488],
 [23255872.24279698, 

In [7]:
x_plus_sol_list

[[-0.9250668448853224, 1.1750190604612505],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0]]

In [8]:
lambda_i_sol_list

[[-0.18890642503614774],
 [-2.7723160079475626],
 [5.544632015895125],
 [-11.08926403179025],
 [22.1785280635805],
 [-44.357056127161],
 [88.714112254322],
 [-177.4282245086439],
 [354.85644901728756],
 [-709.7128980345756],
 [1419.4257960691507],
 [-2838.8515921383027],
 [5677.703184276606],
 [-11355.406368553211],
 [22710.812737106426],
 [-45421.625474212844],
 [90843.2509484257],
 [-181686.50189685138],
 [363373.0037937028],
 [-726746.0075874055],
 [1453492.0151748112],
 [-2906984.030349622],
 [5813968.060699245],
 [-11627936.121398488],
 [23255872.24279698],
 [-46511744.48559395],
 [93023488.97118792],
 [-186046977.9423758],
 [372093955.8847517],
 [-744187911.7695032],
 [1488375823.5390067],
 [-2976751647.078013],
 [5953503294.156027],
 [-11907006588.312052],
 [23814013176.6241],
 [-47628026353.248184],
 [95256052706.49635],
 [-190512105412.9928],
 [381024210825.9855],
 [-762048421651.9706],
 [1524096843303.9404],
 [-3048193686607.879],
 [6096387373215.754],
 [-12192774746431.5],
 